In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor, Ridge, Lasso
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
house_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
house_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

# I'll convert ordinal variables to numeric, don't bother in the next cell it's just convert categorical data to numerical to represent it in the right form.

In [ ]:
#Assigning numerical values for nominial train data
Numlist1 = ['BsmtQual','BsmtCond', 'FireplaceQu','GarageQual', 'GarageCond']
Numlist2 = ['BsmtExposure']
Numlist3 = ['BsmtFinType1' , 'BsmtFinType2']
Numlist4 = ['PoolQC']
Numlist5 = ['Fence']
Numlist6 = ['ExterQual','ExterCond','HeatingQC', 'KitchenQual']
Numlist7 = ['LotShape']
Numlist8 = ['LandSlope']
Numlist9 = ['Functional']
Numlist10 = ['GarageFinish']
def numeric_map1(x):
    return x.map({'Po' : 1 ,'Fa' : 2 , 'TA' : 3 , 'Gd' : 4 , 'Ex' : 5 , np.nan : 0})
def numeric_map2(y):
    return y.map({'No' : 1 ,'Mn' : 2 , 'Av' : 3 , 'Gd' : 4 , np.nan : 0})
def numeric_map3(z):
    return z.map({'Unf' : 1 ,'LwQ' : 2 , 'Rec' : 3 , 'BLQ' : 4 ,'ALQ': 5, 'GLQ': 6, np.nan : 0})
def numeric_map4(a):
    return a.map({'Fa' : 1 ,'TA' : 2 , 'Gd' : 3 , 'Ex' : 4 , np.nan : 0})
def numeric_map5(b):
    return b.map({'MnWw' : 1 ,'GdWo' : 2 , 'MnPrv' : 3 , 'GdPrv' : 4 , np.nan : 0})
def numeric_map6(c):
    return c.map({'Po' : 1 ,'Fa' : 2 , 'TA' : 3 , 'Gd' : 4 , 'Ex' : 5 })
def numeric_map7(d):
    return d.map({'IR3' : 1 ,'IR2' : 2 , 'IR1' : 3 , 'Reg' : 4 })
def numeric_map8(e):
    return e.map({'Sev' : 1 ,'Mod' : 2 , 'Gtl' : 3 })
def numeric_map9(f):
    return f.map({'Sal' : 1 ,'Sev' : 2 , 'Maj2' : 3 ,'Maj1' : 4, 'Mod' : 5, 'Min2' : 6 , 'Min1' : 7, 'Typ' : 8})
def numeric_map10(g):
    return g.map({'Unf' : 1 ,'RFn' : 2 , 'Fin' : 3 , np.nan : 0})
house_train[Numlist1] = house_train[Numlist1].apply(numeric_map1)
house_train[Numlist2] = house_train[Numlist2].apply(numeric_map2)
house_train[Numlist3] = house_train[Numlist3].apply(numeric_map3)
house_train[Numlist4] = house_train[Numlist4].apply(numeric_map4)
house_train[Numlist5] = house_train[Numlist5].apply(numeric_map5)
house_train[Numlist6] = house_train[Numlist6].apply(numeric_map6)
house_train[Numlist7] = house_train[Numlist7].apply(numeric_map7)
house_train[Numlist8] = house_train[Numlist8].apply(numeric_map8)
house_train[Numlist9] = house_train[Numlist9].apply(numeric_map9)
house_train[Numlist10] = house_train[Numlist10].apply(numeric_map10)

#################################################################

house_test[Numlist1] = house_test[Numlist1].apply(numeric_map1)
house_test[Numlist2] = house_test[Numlist2].apply(numeric_map2)
house_test[Numlist3] = house_test[Numlist3].apply(numeric_map3)
house_test[Numlist4] = house_test[Numlist4].apply(numeric_map4)
house_test[Numlist5] = house_test[Numlist5].apply(numeric_map5)
house_test[Numlist6] = house_test[Numlist6].apply(numeric_map6)
house_test[Numlist7] = house_test[Numlist7].apply(numeric_map7)
house_test[Numlist8] = house_test[Numlist8].apply(numeric_map8)
house_test[Numlist9] = house_test[Numlist9].apply(numeric_map9)
house_test[Numlist10] = house_test[Numlist10].apply(numeric_map10)

# Split data for numerical and categorical variable

In [ ]:
train_num = house_train.select_dtypes(exclude = ['object'])
train_cat = house_train.select_dtypes('object')
test_num = house_test.select_dtypes(exclude = ['object'])
test_cat = house_test.select_dtypes('object')

# EDA on numerical variables

- start with numerical variable 

In [ ]:
for i in train_num.columns:
    plt.figure(figsize=(15,8))
    sns.distplot(train_num[i])

Some features aren't numerical like MSSubClass, YrSold (Identifies the type of dwelling involved in the sale.)
so i don't believe that it will be need it

In [ ]:
train_num.drop('MSSubClass', inplace = True, axis = 1)
test_num.drop('MSSubClass', inplace = True, axis = 1)

In [ ]:
train_num.drop('YrSold', inplace = True, axis = 1)
test_num.drop('YrSold', inplace = True, axis = 1)

In [ ]:
train_num.drop('Id', inplace = True, axis = 1)
test_num.drop('Id', inplace = True, axis = 1)

# Let's Cheack the skewness

In [ ]:
skewed_list = []
for i in train_num.columns:
    if abs(train_num[i].skew()) > 0.5:
        skewed_list.append(i)
skewed_list

I'll pick only the non ordinal features, and exclude the SalePrice also then I'll make Log function on them


In [ ]:
skewed_list = ['LotFrontage',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF2',
 'TotalBsmtSF',
 'BsmtUnfSF',
 '2ndFlrSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'HalfBath',
 'KitchenAbvGr',
 'Fireplaces',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'MiscVal',
 ]

# Cheack correlation & Multicorlinarity

In [ ]:
#corr
#corr['SalePrice'].sort_values(ascending = False).abs()

In [ ]:
corr = train_num.corr()
plt.figure(figsize=(25,25))
sns.heatmap(corr, annot = True)
plt.show()

- PoolQC, PoolArea have **0.9** MultiCorrelation
- GarageQual, GarageCond have **0.96** MultiCorrelation
- GarageFinish, GarageCars have **0.88** MultiCorrelation
- FirePlaces,  Functional have **0.86** MultiCorrelation
- GarageYrBlt, YearBuilt have **0.83** MultiCorrelation
- GrLivArea, TotRmsAbvGrd have **0.83** MultiCorrelation
- BsmtFinSF2, BsmtFinType2 have **0.79** MultiCorrelation
- TotalBsmtFinSF, lstFirSF have **0.82** MultiCorrelation
- **Important Note:** I'll drop one of the column in each Multicorrelation situation but i'll drop the one who's have less correlation with the targt variable

In [ ]:
train_num.drop(columns = ['PoolArea','GarageCond','GarageFinish', 'Functional','GarageYrBlt', 'TotRmsAbvGrd','BsmtFinType2','1stFlrSF'], inplace = True, axis = 1)
test_num.drop(columns = ['PoolArea','GarageCond','GarageFinish', 'Functional','GarageYrBlt', 'TotRmsAbvGrd','BsmtFinType2','1stFlrSF'], inplace = True, axis = 1)

In [ ]:
corr2 = train_num.corr()
corr2['SalePrice'].sort_values(ascending = False).abs()

In [ ]:
# Create list for feature enginnering, any feature have less than 20% correlation I'll put it in this list.
fe_list = ['BsmtUnfSF','BsmtCond', 'BedroomAbvGr', 'PoolQC', 'ScreenPorch', 'MoSold', '3SsnPorch', 'ExterCond','BsmtFinSF2',
          'BsmtHalfBath','MiscVal ', 'LowQualFinSF', 'LandSlope', 'OverallCond ', 'EnclosedPorch',  'KitchenAbvGr','Fence' ]

# Preprocessing for numerical variables

In [ ]:
train_num.isnull().sum()

In [ ]:
train_num = train_num.fillna(train_num.median())
test_num = test_num.fillna(test_num.median())

- Take the log for the selected feature 

In [ ]:
train_num[skewed_list] = train_num[skewed_list].apply(lambda i: np.log1p(i))
test_num[skewed_list] = test_num[skewed_list].apply(lambda i: np.log1p(i))

In [ ]:
y = train_num['SalePrice']
train_num = train_num.iloc[:,:-1]
train_num.info()

# what about Categorical Data

In [ ]:
train_cat.isnull().sum()

# I'll Cheack The Dependancey

In [ ]:
def dependency(data,feature):
    
    table = pd.crosstab(data['SalePrice'],data[feature])
    stat, p, dof, excpected = chi2_contingency(table) 

    print(dof)
    significance_level = 0.05
    print("p value: " + str(p)) 

    if p <= significance_level: 
        print( f'{feature}: REJECT NULL HYPOTHESIS, THE VARIABLES ARE DEPENDENT')
        return(feature)
    else: 
        print(f'{feature}: ACCEPT NULL HYPOTHESIS, THE VARIABLES ARE INDEPENDENT')

In [ ]:

for i in train_cat.columns:
    dependency(house_train,i)


In [ ]:
dependent = ['MSZoning','Street','LotConfig','Neighborhood','MasVnrType',
             'Foundation','CentralAir','SaleType','SaleCondition']

In [ ]:
train_cat = pd.get_dummies(train_cat[dependent], drop_first = True)
test_cat = pd.get_dummies(test_cat[dependent], drop_first = True)

In [ ]:
train_num.shape

In [ ]:
fe_list

In [ ]:
train_num_2 = train_num.copy() 
test_num_2 = test_num.copy()

In [ ]:
train_num_2['BsmtUnfSF'] = train_num_2['BsmtUnfSF']**2
test_num_2['BsmtUnfSF'] = test_num_2['BsmtUnfSF']**2

train_num_2['BsmtCond'] = train_num_2['BsmtCond']**2
test_num_2['BsmtCond'] = test_num_2['BsmtCond']**2

train_num_2['BedroomAbvGr'] = train_num_2['BedroomAbvGr']**2
test_num_2['BedroomAbvGr'] = test_num_2['BedroomAbvGr']**2

train_num_2['PoolQC'] = train_num_2['PoolQC']**2
test_num_2['PoolQC'] = test_num_2['PoolQC']**2

train_num_2['ScreenPorch'] = train_num_2['ScreenPorch']**2
test_num_2['ScreenPorch'] = test_num_2['ScreenPorch']**2

train_num_2['MoSold'] = train_num_2['MoSold']**2
test_num_2['MoSold'] = test_num_2['MoSold']**2

train_num_2['3SsnPorch'] = train_num_2['3SsnPorch']**2
test_num_2['3SsnPorch'] = test_num_2['3SsnPorch']**2

train_num_2['ExterCond'] = train_num_2['ExterCond']**2
test_num_2['ExterCond'] = test_num_2['ExterCond']**2

train_num_2['BsmtFinSF2'] = train_num_2['BsmtFinSF2']**2
test_num_2['BsmtFinSF2'] = test_num_2['BsmtFinSF2']**2

train_num_2['BsmtHalfBath'] = train_num_2['BsmtHalfBath']**2
test_num_2['BsmtHalfBath'] = test_num_2['BsmtHalfBath']**2

train_num_2['MiscVal'] = train_num_2['MiscVal']**2
test_num_2['MiscVal'] = test_num_2['MiscVal']**2

train_num_2['LowQualFinSF'] = train_num_2['LowQualFinSF']**2
test_num_2['LowQualFinSF'] = test_num_2['LowQualFinSF']**2

In [ ]:
#test_2.shape
#train_2.shape

In [ ]:
frames = [train_num, train_cat]
frames2 = [test_num, test_cat]

train = pd.concat(frames, axis = 1)
test = pd.concat(frames2, axis = 1)


In [ ]:
frames_2 = [train_num_2, train_cat]
frames2_2 = [test_num_2, test_cat]

train_2 = pd.concat(frames_2, axis = 1)
test_2 = pd.concat(frames2_2, axis = 1)

In [ ]:
train_cat.info()

- Standardize the data

In [ ]:
sc = StandardScaler()
train = sc.fit_transform(train)
test = sc.fit_transform(test)

In [ ]:
train_2 = sc.fit_transform(train_2)
test_2 = sc.fit_transform(test_2)

- Let's look about our target variable

In [ ]:
sns.distplot(y)

- It's Obviously very skewed 

In [ ]:
y = np.log1p(y)
sns.distplot(y)

# LinearRegression

In [ ]:
lr = LinearRegression()
cv_lr = cross_val_score(lr, train,y,cv=10, scoring = 'neg_root_mean_squared_error')
cv_lr.mean()
lr.fit(train,y)

# ElasticNet

In [ ]:
l1_space = np.linspace(0, 1, 50)
param_grid = {'l1_ratio': l1_space}
elastic_net = ElasticNet()
gm_cv = GridSearchCV(elastic_net, param_grid, cv=10, scoring = 'neg_root_mean_squared_error')
gm_cv.fit(train,y)
gm_cv.score(train,y)

# Ridge

In [ ]:
alpha_space = np.linspace(0, 1, 50)
params_grid = {'alpha':alpha_space}
ridge = Ridge()

ridge_cv = GridSearchCV(ridge, params_grid, cv=10, scoring = 'neg_root_mean_squared_error')
ridge_cv.fit(train,y)
ridge_cv.score(train,y)

# Lasso

In [ ]:
alpha_space = np.linspace(0, 1, 50)
params_grid = {'alpha':alpha_space}
lasso = Lasso()

lasso_cv = GridSearchCV(lasso, params_grid, cv=10, scoring = 'neg_root_mean_squared_error')
lasso_cv.fit(train,y)
lasso_cv.score(train,y)

# SGDRegressor

In [ ]:
alpha_space = np.linspace(0, 1, 30)
params_grid = {'alpha':alpha_space}
SGD = SGDRegressor()

SGD_cv = GridSearchCV(SGD, params_grid, cv=10, scoring = 'neg_root_mean_squared_error')
SGD_cv.fit(train,y)
SGD_cv.score(train,y)  

# XGBoost

In [ ]:
xgb_reg = xgb.XGBRegressor(n_estimators = 1000,
                          max_depth = 5,
                          learning_rate = 0.01,
                            reg_alpha = 1,
                              reg_lambda = 0.5)
xgb_reg.fit(train,y)

# SVR

In [ ]:
svr = SVR(kernel = 'linear')
svr.fit(train,y)

In [ ]:
test_id = house_test['Id']

In [ ]:
out = np.expm1(lasso_cv.predict(test))
sub = pd.DataFrame({'id':test_id, 'SalePrice':out})
print(sub.head())
sub.to_csv('Submission.csv',index=False)